# GROUP PROJECT

# Introduction

The data for this project comes from a research Minecraft server operated by a UBC Computer Science group studying how people behave in open-world games. When players join the server, two types of information are recorded. So the first file called “players.csv” contains one row per player and it includes basic account-level details. The second file is called “sessions.csv” and it has one row per play session and it looks at when a player started playing, and when they stopped, and how long the sessions lasted. The datasets on its own describe a part of a player’s activity, and together they provide a full picture of who the players are and how they interact with the game.

One of the goals of the research team is to figure out which kind of players are most likely to subscribe to the project’s email newsletter. Subscription is optional, so it will act as a sign of interest/engagement. If certain patterns in behaviour or account characteristics reliably predict who subscribes, the team can improve their recruitment strategies and plan server resources with more confidence.

In this report, we focus on the question: Which player characteristics and gameplay behaviours best predict whether a player subscribes to the newsletter? 

To answer this, we first combine both datasets and create summary features for each player. Like the total number of sessions, the total playtime, and the typical session length. This will produce a single tidy table where each row is one player and each column is a characteristic. With this structure set, we can further look at the connections between player activity and subscription, and eventually build models that evaluate how well these features can predict subscribing.


In [ ]:
HELLO